Attempts to join the REPD with BM unit data, in order to find the locations of BM units.

In [ ]:
import pandas as pd
from pathlib import Path

%load_ext nb_black

DATA_PATH = Path("../data/locations")

# REPD
This contains wind farm locations

In [ ]:
repd = pd.read_csv(DATA_PATH / "repd-october-2022.csv", encoding="ISO-8859-1")

In [ ]:
repd

In [ ]:
df_repd_wind = repd[repd["Technology Type"].str.contains("Wind")].sort_values(
    by="Installed Capacity (MWelec)", ascending=False
)
df_repd_wind

# BM data
Need to merge one sheet which has nice capacity information, and another which has nice party information - between them these should give us enough to map to the REPD

In [ ]:
b1420 = pd.read_csv(DATA_PATH / "B1420.csv", header=1)
b1420.head()

In [ ]:
b1420 = b1420.set_index("BM Unit ID")

In [ ]:
reg_units = pd.read_csv(DATA_PATH / "reg_bm_units.csv", header=1)

reg_units = reg_units.set_index("BM Unit ID")

In [ ]:
df_bm = pd.merge(reg_units, b1420, left_index=True, right_index=True, how="left")
df_bm["PSR Type"] = df_bm["PSR Type"].str.strip("&ldquo;")

In [ ]:
df_bm.groupby("PSR Type").count()

In [ ]:
df_bm_wind = df_bm[df_bm["PSR Type"].fillna("unknown").str.contains("Wind")]

In [ ]:
df_bm_wind.columns
df_bm_wind = df_bm_wind.sort_values(
    by="Installed Capacity (Nominal Power) (MW)", ascending=False
)

In [ ]:
len(df_bm_wind)

Not enough wind assets here. Turns out this is because many are mistagged as "generation"

In [ ]:
df_bm[df_bm["PSR Type"] == "Generation"]

Merge in the BM Fuel Units sheet for a clearer picture of fuel type

In [ ]:
df_bm_fuel = pd.read_excel(DATA_PATH / "BMUFuelType.xls", header=0)
df_bm = pd.merge( df_bm,df_bm_fuel.set_index("SETT_BMU_ID"), left_index=True, right_index=True)


Much better

In [ ]:
df_bm["FUEL TYPE"].value_counts()

# Dump data for a manual join

In [ ]:
df_bm[df_bm["FUEL TYPE"] == "WIND"].sort_values(
    by="Installed Capacity (Nominal Power) (MW)", ascending=False
).to_excel(DATA_PATH / "bm_units_combined.xlsx")

In [ ]:
df_repd_wind.to_excel(DATA_PATH / "repd_wind_units.xlsx")

# After mapping
Read back in and graph

In [ ]:
from OSGridConverter import grid2latlong, OSGridReference
import httpx
from functools import cache
from typing import Tuple

In [ ]:
@cache
def convert_bng_to_lat_lng(x: float, y: float) -> Tuple[float, float]:
    """Convert British National Grid xy coords to lat/lng"""

    if not (isinstance(x, float) and abs(x) != 0):
        return None, None

    r = httpx.get(
        f"https://webapps.bgs.ac.uk/data/webservices/CoordConvert_LL_BNG.cfc?method=BNGtoLatLng&easting={x}&northing={y}"
    )
    if r.status_code == 200:
        values = r.json()
        return values.get("LATITUDE"), values.get("LONGITUDE")
    else:
        return None, None

In [ ]:
df = pd.read_excel(DATA_PATH / "bm_units_combined.xlsx", sheet_name="Match").set_index(
    0
)

In [ ]:
df[["lat", "lng"]] = df.apply(
    lambda row: convert_bng_to_lat_lng(row["x"], row["y"]), axis=1, result_type="expand"
)

In [ ]:
df.columns

In [ ]:
df[
    [
        "BMU Name",
        "Installed Capacity (Nominal Power) (MW)",
        "Match To Site Name",
        "lat",
        "lng",
    ]
].to_excel(DATA_PATH / "bm_units_with_locations.xlsx")

# Map at last

In [ ]:
df_bm["coordinates"] = df_bm.apply(lambda row: [row["lng"], row["lat"]], axis=1)

In [ ]:
df_bm["coordinates"]

In [ ]:
df_bm[["coordinates", "Installed Capacity (Nominal Power) (MW)"]].dropna(how="any")

In [ ]:
df_bm["size"]

In [ ]:
import pydeck as pdk

df_bm["size"] = df_bm["Installed Capacity (Nominal Power) (MW)"] + 1
# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    df_bm,
    pickable=True,
    opacity=0.6,
    stroked=False,
    filled=True,
    radius_scale=20,
    radius_min_pixels=1,
    radius_max_pixels=200,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius="size",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(latitude=54, longitude=0, zoom=5, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
)

In [ ]:
r.to_html("scatterplot.html")

In [ ]:
import math

SCATTERPLOT_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/bart-stations.json"
df = pd.read_json(SCATTERPLOT_LAYER_DATA)

# Use pandas to calculate additional data
df["exits_radius"] = df["exits"].apply(lambda exits_count: math.sqrt(exits_count))

# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=6,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius="exits_radius",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}\n{address}"})
r.to_html("scatterplot_layer.html")


In [ ]:
df